# Pipeline output creator

this script creates a nice output for the selected locus.
The output will have 8 sequence files, one png image, one metadata information file, one locusinfo text file, one readme

In [ ]:
# First created 18/10/2019
# Made by L-F-S
# At the University Of Trento, Italy

from os import system, chdir
import sys
from np import unique
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies

# parameters:
feature="Cas9" #WARNING!!! CHANGE THIS!!
datadir="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/"

seqid="LPEAHEHG_01071" #name of sequence #sys.argv

matched_rep=Seq("TAATAGTTCTTGGTATGGTATAA")    #sequence matching the repeat
rpt=Seq("GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT")  #matching repeat
matched_rep_start = 14729
matched_rep_stop = 14751
strand=-1
terminator_end=14653-1 # -1, for indexing reasons. it is the stop if strand = -1, the start, if strand = +1
terminator=Seq("CGTAAAGCTCTGACGCCCTGTCCTAGGACAGGGCGTCaTCTTTTTT".upper())

In [2]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy'],
      dtype='object')

In [3]:
tabellazza=pd.read_csv(datadir+"/3tabellazza/crisprcas_hits_table.csv", index_col=0)
tabellazza.columns

Index(['Unnamed: 0.1', 'Genome Name', 'Study', 'Sample Name', 'Assembler',
       'Genome Size', 'N50', '# Contigs', 'Completeness', 'Contamination',
       'Strain Heterogeneity', 'SGB ID', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_CRISPR', 'prokka_cas', 'uniref_cas'],
      dtype='object')

In [5]:
spacers=[]
repeats=[]
repeat_start_pos=[]


cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

dataset, genomename, samplename, s3_dataset, s3_genome = filename_discrepancies.master_filename_discrepancies(dataset,genomename)

# get CRISPR spacer and repeat sequence

mincedCRISPRfilename=datadir+"1crisprsearch/out/"+s3_dataset+"/"+s3_genomename+".fa.minced.out"
f=open(mincedCRISPRfilename, "r")
for line in f.readlines():
    print(line)
    if line[0]== "1" or line[0]== "2" or line[0]== "3" or line[0]== "4" or line[0]== "5" or \
    line[0]== "6" or line[0]== "7" or line[0]== "8" or line[0]== "9" or line[0]== "0":

        repeats.append(line.split("\t")[2])

        repeat_start_pos.append(line.split("\t")[0])

        if not line.split("\t")[3]=="\n":
            spacers.append(line.split("\t")[3])
f.close()

# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai genomename della cas e/o seqid della cas (anzi famo solo seqid, che magari ci sn doppioni in genomi) e ti dà tt il locus

# print start and stop of CRISPR
unostranosplit=cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0].split("\', \'")
crispr_start, crispr_stop = int(unostranosplit[1]), int(unostranosplit[2].strip("\')]\""))

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.

prokka_anno=tabellazza[tabellazza["Genome Name"]==s3_genomename]["prokka_cas"].iloc[0]
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così

prokkaslist=prokka_anno.split(">")
prokkaslist.pop(0)
# get contigname of current effectorcas   
cas_position={}
for casanno in prokkaslist:
    unostranosplit=casanno.split("Name=")
    casnumber=unostranosplit[1][3] #because we know that we are loooking at 'good' loci, this is just going to be 9, 1 or 2
    unaltrostranosplit=casanno.split("\t")
    cas_start=unaltrostranosplit[3]
    cas_stop=unaltrostranosplit[4]
    cas_position["Cas"+casnumber]=[int(cas_start),int(cas_stop)]

genomefilename= "/shares/CIBIO-Storage/CM/scratch/tmp_projects/epasolli_darkmatter/allcontigs/ALLreconstructedgenomes/"+str(SGB)+"/"+genomename+".fa"
print("Retrieving locus")


# access contig:

for record in SeqIO.parse(genomefilename,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, "/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb", "fasta")
        print("CRISPR si trova", crispr_start, crispr_stop)
        print("Cas9 si trova", cas_position["Cas9"])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        cas9start=[int(x) for x in cas_position["Cas9"]][0]
        cas9stop=[int(x) for x in cas_position["Cas9"]][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()
 
  
        print(len(cas9_nnseq)/3,len(cas9_aa))

        # check strand for all Cas9s (scriveresti ben meno codice a metterlo in una funzioncina, ma ok)
        if my_translated_cas9==cas9_aa:
            print("Cas9 locus on plus strand")
            plus=True
        else:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas9 locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)

        #extact a locus, 200 bp a monte e a valle dei cosi che ho:
        starts, stops=[startstop[0]for startstop in cas_position.values()], [startstop[1]for startstop in cas_position.values() ]
        minimo=min([crispr_start]+starts)
        print("start: ", minimo)
        massimo=max([crispr_stop]+stops)
        print("end: ", massimo)

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        blast_folder="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"
        chdir(blast_folder)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")
        for n, repeat in enumerate([Seq(sequence) for sequence in unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db
        print("Running blastn of query file agianst all contigs")

        #3. make db file
        dbfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/tempblastdb"
        system("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")



        blastoutfile="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/7tracrRNA/"\
        +str(SGB)+"__"+seqid+"__"+genomename+"__"+feature+".trcr.blastout"
        blastn_command = "blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11"
        print(blastn_command)
        system(blastn_command)



-----------------------------------------------------
Cas9 id:	LPEAHEHG_01071
SGB:	15299
Genome Name:	 ChengpingW_2017__AS43raw__bin.51
Contig:	NODE_190_length_65757_cov_25.3876 
-----------------------------------------------------

Sequence 'NODE_190_length_65757_cov_25.3876' (65757 bp)



CRISPR 1   Range: 9802 - 10102

POSITION	REPEAT				SPACER

--------	------------------------------------	------------------------------

9802		GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT	CTTCCGTGCCATTTCCTCTATGACAGAATC	[ 36, 30 ]

9868		GTCATAGTTCCCTAACAGCTCTTGGTATGGTATAAT	CTGCCCGGCGACGTGCGTGTGAGCGATACG	[ 36, 30 ]

9934		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	CCTGCCGCCCAATGCCCAGGACAACGAACT	[ 36, 30 ]

10000		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	AACGACTATTCAACGCACGGTAATAGCGCCT	[ 36, 31 ]

10067		GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT	

--------	------------------------------------	------------------------------

Repeats: 5	Average Length: 36		Average Length: 30



Time to find repeats: 106 ms





Retrieving l